# Hi!ckaton
This notebook presents the work the group n°12 has been producing.

In [14]:
# necessary imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

In [15]:
# load the training dataset
df_train =  pd.read_csv("hfactory_magic_folders/water_shortage_prediction/X_train_Hi5.csv")

/tmp/ipykernel_21334/2993170361.py:2: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train =  pd.read_csv("hfactory_magic_folders/water_shortage_prediction/X_train_Hi5.csv")


In [16]:
# first exploration of the dataset
rows, columns = df_train.shape

print(f"Number of rows : {rows}")
print(f"Number of columns : {columns}")

Number of rows : 2830316
Number of columns : 136


In [ ]:
usefull_columns = [ 'piezo_station_altitude',
                   'piezo_station_longitude',
                   'piezo_station_latitude',
                   'piezo_station_bss_id',
                   'piezo_station_investigation_depth',
                   'piezo_measurement_date', # To process
                   'piezo_qualification', # To process (discrimination)
                   'meteo_rain_height', # High impact
                   'meteo_temperature_avg',
                   'hydro_observation_result_elab',
                   'hydro_hydro_quantity_elab', # To process, according to its units
                   'piezo_groundwater_level_category']
                   
label_column = 'piezo_groundwater_level_category'

df_train = df_train[usefull_columns]
df_train.shape

(2830316, 11)

The database is highly time dependent, that means that the prediction of the next days for one station, will surely rely on the previous days. Thus, we can add some 'lag features', these will be the height rain for the last 10 days.

In [ ]:
df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date'])
curr_id = df_train['piezo_station_bss_id'][0]
past_measure = [(df_train['piezo_measurement_date'][0],df_train['meteo_rain_height'][0])]

for i in range(1,df_train.shape[0]):
    if df_train['piezo_station_bss_id'][i] == curr_id:
        for j in range(max(0,len(past_measure) - 10), len(past_measure)):

            stmp = (df_train['piezo_measurement_date'][i] - past_measure[j][0]).days
            if 0 < stmp < 10:
                df_train[f'piezo_measurement_day_-{stmp}'][i] = past_measure[j][1]
        past_measure.append((df_train['piezo_measurement_date'][i],df_train['meteo_rain_height'][i]))
                
    else:
        curr_id = df_train['piezo_station_bss_id'][i]
        past_measure = [(df_train['piezo_measurement_date'][i],df_train['meteo_rain_height'][i])]
    if i%100000 == 0:
        print(i)

In [ ]:
df_train = df_train.drop(columns = ['piezo_station_bss_id'])

In [18]:
df_train.shape

(2830316, 11)

In [ ]:
# The dates (drop the day information and create a month and a year column)
df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date'])

df_train['day'] = df_train['piezo_measurement_date'].dt.day 
df_train['month'] = df_train['piezo_measurement_date'].dt.month
df_train['year'] = df_train['piezo_measurement_date'].dt.year
df_train = df_train.drop(columns = ['piezo_measurement_date']) 

In [20]:
# The quality of the measure (drop the uncertain or wrong ones)
df_train.drop(df_train[df_train['piezo_qualification'] == 'Incorrecte'].index, inplace=True)
df_train.drop(df_train[df_train['piezo_qualification'] == 'Incertaine'].index, inplace=True)
df_train = df_train.drop(columns = ['piezo_qualification']) 

In [7]:
df_train.shape

(2812068, 13)

In [22]:
df_train['hydro_observation_result_elab'].describe()

count    2.812068e+06
mean     1.316905e+05
std      9.938560e+06
min     -1.103870e+05
25%      3.930000e+02
50%      1.539000e+03
75%      7.755000e+03
max      9.999999e+08
Name: hydro_observation_result_elab, dtype: float64

In [23]:
# Adapt the value of hydro_hydro_quantity_elab with the right unit
df_train['hydro_observation_result_elab'] = df_train['hydro_observation_result_elab'] / 30
df_train = df_train.drop(columns = ['hydro_hydro_quantity_elab']) 

In [24]:
df_train['hydro_observation_result_elab']

0             61.100000
1           3112.966667
2           3607.900000
3          10182.133333
4              2.833333
               ...     
2830311      173.166667
2830312       36.300000
2830313       36.300000
2830314       10.433333
2830315        3.100000
Name: hydro_observation_result_elab, Length: 2812068, dtype: float64

In [25]:
df_train.columns

Index(['piezo_station_altitude', 'piezo_station_longitude',
       'piezo_station_latitude', 'piezo_station_investigation_depth',
       'meteo_rain_height', 'meteo_temperature_avg',
       'hydro_observation_result_elab', 'piezo_groundwater_level_category',
       'day', 'month', 'year'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns', None)
df_train.head(10)

,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_station_investigation_depth,meteo_rain_height,meteo_temperature_avg,hydro_observation_result_elab,piezo_groundwater_level_category,day,month,year
0,232.00,5.748241,45.828148,20.00,0.2,0.7,61.100000,High,1,1,2020
1,247.25,5.356637,46.028102,35.60,0.0,-1.2,3112.966667,Very High,1,1,2020
2,218.77,5.220795,45.895734,35.22,NaN,-0.6,3607.900000,High,1,1,2020
3,499.85,5.948977,46.201180,34.20,0.0,0.4,10182.133333,Very High,1,1,2020
4,260.00,5.313353,46.136402,37.30,0.0,-0.8,2.833333,Very Low,1,1,2020
5,250.00,5.679690,45.874738,50.00,0.0,3.5,61.100000,Very High,1,1,2020
6,208.21,5.285156,45.837397,21.00,0.0,-1.2,3607.900000,High,1,1,2020
7,558.68,6.069710,46.329030,78.00,0.0,0.5,20.500000,Very Low,1,1,2020
8,250.45,5.823274,45.919630,10.00,0.0,-0.4,14373.466667,Low,1,1,2020
9,265.22,4.830754,46.018341,62.60,0.0,-0.3,10.866667,High,1,1,2020


In [29]:
df_train.to_csv('X_train.csv')